In [17]:
#categorize all pixels in the bird's view into left group and right group
#fit left and right quadratic curve lines using detected pixels in left group and right group respectively

import cv2  
import numpy as np
import math


def find_lane(frame):
    y,x=frame.shape
    #print(frame.shape)
    nonzero=np.transpose(np.nonzero(frame))
    L=[m for m in nonzero if m[1]<0.5*x]
    R=[n for n in nonzero if n[1]>0.5*x]    
    #print('L',L)
    #print('R',R)
    return L,R
    
     

def fit_lanes(points_l, points_r, fit_globally=True):
    yl,xl=np.transpose(points_l)
    yr,xr=np.transpose(points_r)
    fit_ret={}
    if fit_globally:
        parameters = np.polyfit(yl, xl, 2)
        fit_ret.update({'al':parameters[0], 'bl':parameters[1],'cl':parameters[2],
                       'ar':parameters[0], 'br':parameters[1],'cr':parameters[2]})
    else:
        parameters_l= np.polyfit(yl, xl, 2)
        parameters_r= np.polyfit(yr, xr, 2)
        fit_ret.update({'al':parameters_l[0], 'bl':parameters_l[1],'cl':parameters_l[2],
                       'ar':parameters_r[0], 'br':parameters_r[1],'cr':parameters_r[2]})

    return fit_ret

def draw_lanes(frame,parameters):
    pts_l=[]
    pts_r=[]
    y=np.linspace(0,frame.shape[0]-1,num=frame.shape[0])
    xl=parameters['al']*y**2+parameters['bl']*y+parameters['cl']
    xr=parameters['ar']*y**2+parameters['br']*y+parameters['cr']
    pts_l=np.array([np.transpose(np.vstack([xl,y]))])
    pts_r=np.array([np.flipud(np.transpose(np.vstack([xr,y])))])
    polygon = np.zeros_like(frame)
    pts = np.hstack((pts_l, pts_r))
    cv2.fillPoly(polygon, np.int_([pts]), (255,0,0))
    return polygon


#run the test video
cap = cv2.VideoCapture('project_video.mp4',0)
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('outcome.avi',fourcc, 20.0, (1280,720))
frame_count = 1

while(cap.isOpened()):
    _,frame = cap.read()  
    #print('frame',frame.shape)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(3,3),0)
    edges_white=cv2.Canny(gray,300,350)
    
    HSL = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS)
    lower = np.uint8([ 10,   0,  100])
    upper = np.uint8([ 40, 255, 255])
    yellow_mask = cv2.inRange(HSL, lower, upper)
    YellowImg = cv2.bitwise_and(frame, frame, mask=yellow_mask)
    GrayImg = cv2.cvtColor(YellowImg, cv2.COLOR_BGR2GRAY)
    blurredImg = cv2.GaussianBlur(GrayImg, (3, 3), 0)
    edges_yellow = cv2.Canny(blurredImg, threshold1=100, threshold2=210)
    
    edges=cv2.addWeighted(edges_white, 1.0, edges_yellow, 1.0,0)
    
    
    original=frame.copy()
    
    
    original=frame.copy()
    n1,n2,n3=original.shape
    
    # implement homography transformation
    #src = np.float32([[237, 720],[1100, 720],[440, 450],[840, 450]])#multilanes view 
    #dst = np.float32([[437, 720],[800, 720],[300, 0],[980, 0]])#multilanes view
    src = np.float32([[200, 720],[1100, 720],[590, 450],[685, 450]])#singlelanes view
    dst = np.float32([[150, 720],[1020, 720],[300, 0],[980, 0]])#singlelanes view
    #src = np.float32([[200, 720],[1100, 720],[595, 450],[685, 450]])#singlelanes view
    #dst = np.float32([[300, 720],[980, 720],[300, 0],[980, 0]])#singlelanes view
    world=np.array(src)
    ground=np.array(dst)
    h,status=cv2.findHomography(world,ground,cv2.RANSAC,5,maxIters=100, confidence=0.99)
    h_inv,status=cv2.findHomography(world,ground,cv2.RANSAC,5,maxIters=100, confidence=0.99)#h_inv for unwarpping the bird's view 
    projected_transformation=cv2.warpPerspective(original,h,(n2,n1),flags=cv2.INTER_LINEAR)
    projected_edges=cv2.warpPerspective(edges,h,(n2,n1),flags=cv2.INTER_LINEAR)
    projected_edges_copy=cv2.cvtColor(projected_edges,cv2.COLOR_GRAY2BGR)
    cv2.line(projected_edges_copy,(int(0.5*n2),0),(int(0.5*n2),n1),(0,0,255),3)

    
   
    points=find_lane(projected_edges)                                #find lanes in bird's view
    parameters=fit_lanes(points[0], points[1], fit_globally=False)   #fit the quadratic curve model and get curve parameters
    #print('parameters=',parameters)
    polygon=draw_lanes(original,parameters)                          #draw curve lines using the calculated parameters
   
    
    
    #projected back to the world view
    h,status=cv2.findHomography(ground,world,cv2.RANSAC,5,maxIters=100, confidence=0.99)
    h_inv,status=cv2.findHomography(ground,world,cv2.RANSAC,5,maxIters=100, confidence=0.99)#h_inv for unwarpping the bird's view 
    bird_view=cv2.addWeighted(projected_transformation, 1.0, polygon, 0.5,0)
    polygon_back=cv2.warpPerspective(polygon,h,(n2,n1),flags=cv2.INTER_LINEAR)
    outcome=cv2.addWeighted(original, 1.0, polygon_back, 0.5,0)# add the polygon layer to the oringinal frame
    
    
    #cv2.imshow("canny",edges)
    cv2.imshow('projected edge', projected_edges_copy)
    #cv2.imshow('projected transformation',projected_transformation)
    #cv2.imshow('polygon',polygon)
    cv2.imshow('bird view',bird_view)
    cv2.imshow('outcome',outcome)

    
    #out.write(outcome)
    
    params = []  
    #params.append(cv.CV_IMWRITE_PXM_BINARY)  
    params.append(1)  
    #cv2.imwrite("world" + "_%d.jpg" % frame_count, out, params)
    cv2.imwrite('projected edge' + "_%d.jpg" % frame_count, projected_edges_copy, params)
    #cv2.imwrite('bird view' + "_%d.jpg" % frame_count, bird_view, params)
    #cv2.imwrite('outcome' + "_%d.jpg" % frame_count, outcome, params)
    frame_count = frame_count + 1  
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
cap.release()
out.release()
cv2.destroyAllWindows()